In [135]:
select_example = input('Path or Example:')
FILE_PATH = None
SAMPLES = {
    1: 'KOAX_20240521_1844',
    2: 'KOAX_20240521_1849',
    3: 'KOAX_20240523_2138',

}
if select_example.isdigit():
    FILE_PATH = f"{'./Samples/'}{SAMPLES[int(select_example)]}" # I don't care if it is out of range, I just want to test the code
else:
    FILE_PATH = select_example
import os
fsize = os.stat(FILE_PATH).st_size
f = open(FILE_PATH, 'rb')
import bz2
import namedstruct as ns
import logging

In [136]:
f.seek(0)

log = logging.getLogger(__name__)

VHR = ns.nstruct(
    (ns.char[6],'Tape'),            # Magic number: AR2V00
    (ns.char[2],'Version'),         # Version number
    (ns.char[1],'End of Tape'),     # End of Tape marker
    (ns.char[3],'Sequence Number'), # Extension number
    (ns.uint32,'Date_d'),           # Date of Volume, days since 1/1/1970
    (ns.uint32,'Time_ms'),          # Time of Volume, milliseconds since midnight
    (ns.char[4],'Station'),         # Station ID (ICAO)
    name='VHR',padding=1,endian='>')
LDM_CR = ns.nstruct(
    (ns.int32,'ctrl_word'),      # Control word - lenth of the compressed data
    (ns.raw,'data'),                       # This data is bz2 compressed when saved
    name='LDM_CR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word


L2A = ns.nstruct(
    (VHR,'VHR'),                       # Volume Header Record
    (LDM_CR[0],'records'),             # LDM_CR records 
    name='L2A',padding=1,endian='>')

# test = VHR.parse(f.read(24))
fi, __ = L2A.parse(f.read())
# go back to the first LDM_CR
f.seek(24)
while(f.tell() < fsize): # read each LDM_CR into a record and add it to the archive - we do this since the datatype is raw here.
    l = f.tell()
    s = int.from_bytes(f.read(4))
    if(s==b'\xff\xff'):
        break
    f.seek(l)
    d = f.read(s+4)
    if (d == b''):
        log.debug('bad data')
    ldm = LDM_CR.create(d)
    fi.records.append(ldm)


In [139]:

if True:
    f.close() # close the file
msg_header_channel_enum = ns.enum('msg_header_channel_enum', None, ns.uint8, bitwise=False,
                    # Note: if both redundent channels are 0s, single channel mode. 
                           RC_1 = 0,    # redundant channel 1
                           RC_2 = 1,    # redundant channel 2
                           unused = 2,  # unused. 
                           ORDA = 3     # ORDA channel (vs legacy)
                           )
# Message header structure 
msg_header = ns.nstruct(
        (ns.uint32[3],'prepad'),                # padding
        (ns.uint16,'message_size'),             # size of message in halfwords
        (msg_header_channel_enum,'channel'),    # actually bit field
        (ns.uint8,'message_type'),              # type of message
        (ns.uint16,'sequence'),                 # sequence number of message
        (ns.uint16,'julian_date'),              # days since 1/1/1970
        (ns.uint32,'milliseconds'),             # milliseconds since midnight
        # Remember, is these 2 are not 1, then the message is extended and needs to be combined with the next message
        (ns.uint16,'segments'),                 # number of segments in message  
        (ns.uint16,'segment'),                  # segment number of this message
        name='msg_header', padding=1, prepack=ns.packvalue(2432,'message_size'), endian='>') # TODO: Use size of message to determine size of message 

# Base message structure
msg_base = ns.nstruct(
    (msg_header,'header'),
    (ns.raw,'data'),
    name='msg_base',padding=1,endian='>')

# UNCOMPRESSED RECORD data
LDM_UR = ns.nstruct(
    (msg_base[0],'msg'),                # Messages - not sure that this will work since it will hold extended data
    name='LDM_UR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word


ldm = LDM_UR.create('')
for cr in fi.records:
    if(cr.ctrl_word < 0):
        break
    data = bz2.decompress(cr.data[:cr.ctrl_word])
    if data == b'':
        log.error(f"Error decompressing data. CW: {cr.ctrl_word}")
        break
    position = 0
    while(position<len(data)):
        msg_size = int.from_bytes(data[position+12:position+14]) # get size in halfwords
        if(msg_size > 65535):
            log.error(f"Error: Message size too large: {msg_size}")
            break
        elif(msg_size<9):
            log.info(f"Error: Message size too small: {msg_size}")
            break
        else:
            log.debug(f"Message size: {msg_size}")
            #TODO: parse the message into the correct message type instead of raw data
            a_record_that_exists = msg_base.create(data[position:position+msg_size])
            ldm.msg.append(a_record_that_exists)
            position+=msg_size

for a in ldm.msg:
    # print("",end='')
    print(f"{a.header.message_type}:\t{a.header.milliseconds}")



15:	47712310
1:	65537
0:	16777473
31:	67777966
16:	18678053
31:	67779614
139:	27131904
31:	67782302
31:	67784996
141:	10420385
146:	9633957
31:	67787729
154:	10092713
0:	2432734976
1:	2936119297
168:	28901794
171:	27328933
213:	27918759
1:	3137339394
31:	67790423
16:	18219281
1:	1828810753
155:	2459931292
31:	67795121
31:	67797223
31:	67800320
106:	16843009
31:	67803468
31:	67806578
145:	16843155
148:	2257294482
115:	1903324530
31:	67809692
31:	67814620
31:	67816267
31:	67818955
31:	67821670
144:	8913028
0:	2533397248
227:	16726783
31:	67824387
31:	67827070
173:	15532138
31:	67831725
31:	67833817
31:	67836917
31:	67840020
167:	16843009
157:	2610996129
116:	2138669175
31:	67843131
31:	67846240
31:	67851247
141:	7995670
31:	67852749
31:	67855227
31:	67857690
31:	67860183
55:	22806873
31:	67862656
180:	11337915
209:	13631646
0:	0
31:	67867196
31:	67869321
110:	16843009
12:	1073762898
31:	67872411
140:	16843009
133:	2223211395
31:	67875549
66:	16843009
136:	2273675140
31:	67878642
31:	6788